In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
!sudo apt-get --purge remove "*cublas*" "cuda*" "nsight*"
!sudo apt-get --purge remove "*nvidia*"

In [ ]:
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/debian11/x86_64/ /"
!sudo add-apt-repository contrib
!sudo apt-get update
!sudo apt-get -y install cuda-11-3

In [ ]:
!wget https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz
!sudo apt install cmake
!tar xJvf jumanpp-2.0.0-rc3.tar.xz
!cd jumanpp-2.0.0-rc3/
!mkdir bld
!cd bld
!cmake ..
!sudo make install

In [ ]:
!pip install pyknp

In [ ]:
!git lfs install
!git clone https://huggingface.co/megagonlabs/roberta-long-japanese
GIT_LFS_SKIP_SMUDGE=1

In [ ]:
CUDA_LAUNCH_BLOCKING=1

In [ ]:
!pip install --quiet transformers==4.18.0
!pip install --quiet tokenizers==0.12.1
!pip install --quiet sentencepiece
!pip install --quiet japanize-matplotlib
!pip install transformers fugashi ipadic >> /dev/null

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import japanize_matplotlib

import torch
import transformers

from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import sys
import os
import re
import random

from time import time
from tqdm import tqdm

from contextlib import contextmanager
import lightgbm as lgb

import re
import requests
import unicodedata
import nltk
from nltk.corpus import wordnet
from bs4 import BeautifulSoup
nltk.download(['wordnet', 'stopwords', 'punkt'])

In [ ]:
pd.set_option('max_rows', 400)
pd.set_option('max_columns', 1000)

In [ ]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

seed_everything(42)

In [ ]:
INPUT = "/content/drive/MyDrive/nishika/" # 所望のディレクトリに変更してください。
train_image_path = "/content/drive/MyDrive/nishika/train/"
test_image_path = "/content/drive/MyDrive/nishika/test/"

In [ ]:
train_df = pd.read_csv(os.path.join(INPUT, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT, "test.csv"))
submission_df = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))

In [ ]:
print(f"train_data: {train_df.shape}")
display(train_df.head())

print(f"test_data: {test_df.shape}")
display(test_df.head())

In [ ]:
from transformers import AutoTokenizer, AutoModel

class RobertaSequenceVectorizer:
    def __init__(self, model_name: str):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model_name = model_name
        self.tokenizer =AutoTokenizer.from_pretrained(self.model_name)
        #self.tokenizer.do_lower_case = True 
        self.roberta_model =AutoModel.from_pretrained(self.model_name)
        self.roberta_model = self.roberta_model.to(self.device)
        self.max_len = 512


    def vectorize(self, sentence : str) -> np.array:
        inp = self.tokenizer.encode(sentence)
        len_inp = len(inp)

        if len_inp >= self.max_len:
            inputs = inp[:self.max_len]
            masks = [1] * self.max_len
        else:
            inputs = inp + [0] * (self.max_len - len_inp)
            masks = [1] * len_inp + [0] * (self.max_len - len_inp)

        inputs_tensor = torch.tensor([inputs], dtype=torch.long).to(self.device)
        masks_tensor = torch.tensor([masks], dtype=torch.long).to(self.device)

        roberta_out = self.roberta_model(inputs_tensor, masks_tensor)
        seq_out = roberta_out['last_hidden_state']

        if torch.cuda.is_available():    
            return seq_out[0][0].cpu().detach().numpy()
        else:
            return seq_out[0][0].detach().numpy()

In [ ]:
RSV = RobertaSequenceVectorizer("megagonlabs/roberta-long-japanese")

## テキストの欠損値を補間します
train_df["text"] = train_df["text"].fillna('NaN')
test_df["text"] = test_df["text"].fillna('NaN')

## BERT特徴量 
features_text_train = np.stack(train_df["text"].fillna("").map(lambda x: RSV.vectorize(x).reshape(-1)).values)
features_text_test = np.stack(test_df["text"].fillna("").map(lambda x: RSV.vectorize(x).reshape(-1)).values)

In [ ]:
## テキスト特徴量
features_text_train_df = pd.DataFrame(features_text_train).add_prefix("long-roberta-text")
features_text_test_df = pd.DataFrame(features_text_test).add_prefix("long-roberta-text")

train_df = pd.concat([train_df, features_text_train_df], axis=1)
test_df = pd.concat([test_df, features_text_test_df], axis=1)

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df["is_laugh"].value_counts()

In [ ]:
train_df.to_csv('embedding_train_text_long_roberata.csv')

In [ ]:
test_df.to_csv('embedding_test_text_long_roberta.csv')